In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

In [2]:
df = pd.read_csv("/content/Rainfall_Data_LL.csv")

In [3]:
df1 = df.loc[df['SUBDIVISION'] == 'Vidarbha'].iloc[:, 2:16]
df2 = pd.melt(df1, id_vars='YEAR', value_vars=df1.columns[1:-1])
df2['Date'] = df2['variable'] + ' ' + df2['YEAR'].astype(str)
df2.loc[:,'Date'] = df2['Date'].apply(lambda x : datetime.strptime(x, '%b %Y'))
df2.columns=['Year', 'Month', 'Rainfall', 'Date']
df2.sort_values(by='Date', inplace=True)
df3 = df2.drop(columns = ["Month", "Year"])
df3.set_index("Date", inplace = True)

In [4]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.1 MB/s eta 0:00:00


In [5]:
from pmdarima import auto_arima
train = df3.iloc[:-90]
test = df3.iloc[-90:]

In [6]:
model4 = auto_arima(y = df3.Rainfall, m = 12)

In [7]:
predictions3 = pd.Series(model4.predict(n_periods = 12))
predictions3

2018-01-01      9.286351
2018-02-01      7.877742
2018-03-01     20.391922
2018-04-01      9.114600
2018-05-01      9.785673
2018-06-01    171.246806
2018-07-01    324.358146
2018-08-01    260.696226
2018-09-01    174.866918
2018-10-01     50.230277
2018-11-01     11.463384
2018-12-01      4.641299
Freq: MS, dtype: float64

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pip install bz2file

  Preparing metadata (setup.py) ... done
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6868 sha256=6580174ac70b42b0a0e02ab4428c68e0a74cc7375f15c77920292d7b54db6608
  Stored in directory: /root/.cache/pip/wheels/69/ee/f7/6fccd10cb65421ba2da64fa6caf8ee7fbae0059884af8c8587
Successfully built bz2file


In [10]:
import bz2file as bz2
import pickle

In [11]:
def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2', 'w') as f:
    pickle.dump(data, f)

In [12]:
compressed_pickle('model4', model4)

In [13]:
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = pickle.load(data)
  return data

In [14]:
model = decompress_pickle('model4.pbz2')

In [15]:
model.predict(n_periods = 12)

2018-01-01      9.286351
2018-02-01      7.877742
2018-03-01     20.391922
2018-04-01      9.114600
2018-05-01      9.785673
2018-06-01    171.246806
2018-07-01    324.358146
2018-08-01    260.696226
2018-09-01    174.866918
2018-10-01     50.230277
2018-11-01     11.463384
2018-12-01      4.641299
Freq: MS, dtype: float64